In [10]:
import igraph
from igraph import Graph, EdgeSeq


nr_vertices = 25
v_label = list(map(str, range(nr_vertices)))
G = Graph.Tree(nr_vertices, 2) # 2 stands for children number
lay = G.layout('rt')

position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

es = EdgeSeq(G) # sequence of edges
E = [e.tuple for e in G.es] # list of edges

L = len(position)
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

labels = v_label

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))

In [3]:
class Node:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = []

# Build a small tree
root = Node()
child1 = Node(root)
child2 = Node(root)
root.children = [child1, child2]
grandchild = Node(child1)
child1.children = [grandchild]

# Assign indices and gather edges
nodes = []
edges = []
def traverse(node, parent_idx=None):
    idx = len(nodes)
    nodes.append(node)
    if parent_idx is not None:
        edges.append((parent_idx, idx))
    for c in node.children:
        traverse(c, idx)

traverse(root)


G = Graph(edges=edges, directed=False)
lay = G.layout('rt')
# Convert positions to Plotly points
Xn, Yn, Xe, Ye = [], [], [], []
for i, pos in enumerate(lay):
    Xn.append(pos[0])
    Yn.append(pos[1])
for e in G.es:
    Xe += [lay[e.source][0], lay[e.target][0], None]
    Ye += [lay[e.source][1], lay[e.target][1], None]

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe, y=Ye, mode='lines'))
fig.add_trace(go.Scatter(x=Xn, y=Yn, mode='markers'))
fig.update_yaxes(autorange='reversed')
fig.show()

In [26]:
import igraph
import plotly.graph_objects as go
from igraph import EdgeSeq, Graph


class TreePlot:
    def __init__(self, root: Node):
        self.root = root
        self.nodes = []
        self.edges = []

    def create_tree(self):
        def traverse(node: Node, parent_idx=None):
            idx = len(self.nodes)
            self.nodes.append(node)
            if parent_idx is not None:
                self.edges.append((parent_idx, idx))
            for c in node.children:
                traverse(c, idx)

        traverse(self.root)
        self.create_plot()

    def create_plot(self):
        G = Graph(edges=self.edges, directed=False)
        lay = G.layout("rt")
        # Convert positions to Plotly points
        Xn, Yn, Xe, Ye = [], [], [], []
        for i, pos in enumerate(lay):
            Xn.append(pos[0])
            Yn.append(pos[1])
        for e in G.es:
            Xe += [lay[e.source][0], lay[e.target][0], None]
            Ye += [lay[e.source][1], lay[e.target][1], None]

        import plotly.graph_objects as go

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=Xe, y=Ye, mode="lines"))
        fig.add_trace(go.Scatter(x=Xn, y=Yn, mode="markers"))
        fig.update_yaxes(autorange="reversed")
        fig.show()

In [29]:
root = Node()
child1 = Node(root)
child2 = Node(root)
root.children = [child1, child2]
grandchild = Node(child1)
child1.children = [grandchild]


tp = TreePlot(root)
tp.create_tree()


In [41]:
def rotate_state(state: list[str]) -> list[str]:
    rotated_state: list[str] = []

    for i in range(len(state)):
        tmp = ""
        for j in range(len(state)):
            tmp += state[j][i]

        rotated_state.append(tmp)
    rotated_state.reverse()
    return rotated_state


def beatify_state(state: list[str]) -> str:
    beautified_state: str = ""
    for i in range(len(state)):
        for j in range(len(state)):
            beautified_state += f"{state[i][j]} "
        beautified_state += "<br>"
    return beautified_state


rot = rotate_state(["..#.#", "..X..", ".O.X#", "..XO.", "....."])
beatify_state(rot)

'# . # . . <br>. . X O . <br># X . X . <br>. . O . . <br>. . . . . <br>'

In [43]:
import torch

a = torch.tensor(
    [
        -1.2114e-02,
        7.4093e-03,
        -2.2767e-02,
        5.7563e-03,
        -2.1125e-02,
        2.0868e-02,
        4.5380e-03,
        5.8539e-03,
        -9.1735e-03,
        3.5187e-02,
        5.7606e-03,
        5.9224e-03,
        -1.1780e-02,
        -1.6311e-02,
        2.9002e-02,
        -8.8655e-03,
        -1.0327e-02,
        1.2472e-02,
        -3.1732e-03,
        4.2059e-02,
        -1.4111e-02,
        -1.6802e-02,
        -1.0000e09,
        2.0477e-02,
        -2.7329e-02,
        -4.4374e-02,
    ],
    device="cuda:0",
)

b = torch.tensor(
    [
        0.0395,
        0.0403,
        0.0391,
        0.0403,
        0.0392,
        0.0409,
        0.0402,
        0.0403,
        0.0397,
        0.0415,
        0.0403,
        0.0403,
        0.0396,
        0.0394,
        0.0412,
        0.0397,
        0.0396,
        0.0405,
        0.0399,
        0.0417,
        0.0395,
        0.0394,
        0.0000,
        0.0409,
        0.0389,
        0.0383,
    ],
    device="cuda:0",
)

print(torch.argmax(a))
print(torch.argmax(b))
print(torch.argmin(a))
print(torch.argmin(b))

tensor(19, device='cuda:0')
tensor(19, device='cuda:0')
tensor(22, device='cuda:0')
tensor(22, device='cuda:0')


In [50]:
import numpy as np

a = np.array([7])
a = a[0]
a == 0 and 4 == 24

np.False_

In [ ]:
import timeit
import numpy as np

board_width = 19
state = [np.array([0,0,0,0,0]), np.array([0,0,0,0,0]), np.array([0,0,0,0,0]), np.array([0,0,0,0,0]), np.array([0,0,0,0,0])]
leng = len(state)

print(timeit.timeit(lambda: len(state) ** 2, number=10**6))
print(timeit.timeit(lambda: len(state) * len(state), number=10**6))
print(timeit.timeit(lambda: leng ** 2, number=10**6))
print(timeit.timeit(lambda: leng * leng, number=10**6))

0.044347900002321694
0.0429260000018985
0.03558769999654032
0.025559800000337418


In [ ]:
import torch

a = [1,2,3,4]
a.append(5)


b = torch.zeros(26, device="cuda:0")
b

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [1]:
a = set([1,2,3,4,5,6,76])

for b in a:
    print(b)

1
2
3
4
5
6
76
